In [5]:
import pymysql
import pandas as pd

# 資料庫連線設定
host = 'host.docker.internal'
port = 3306
user = 'root'
passwd = 'password'
db = 'movie'
charset = 'utf8mb4'

# 建立連線
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)
print('Successfully connected!')

# 建立 cursor
cursor = conn.cursor()

# SQL 查詢
query = """
SELECT
    T.tmdb_id, 
    T.tmdb_title, 
    T.country_name,
    T.movie_release_date_min as theater_release, 
    S.movie_release_date_min as streaming_release
FROM
(
    SELECT 
        A.tmdb_id, 
        B.tmdb_title, 
        A.movie_release_date_min, 
        A.country,
        C.country_name
    FROM
    (
        SELECT tmdb_id, country, MIN(movie_release_date) AS movie_release_date_min 
        FROM movie_release
        WHERE movie_type = 3
        GROUP BY tmdb_id, country
    ) A
    LEFT JOIN ( SELECT tmdb_id, tmdb_title FROM movie_name GROUP BY tmdb_id, tmdb_title ) B
    ON A.tmdb_id = B.tmdb_id
    INNER JOIN movie_country C
    ON A.country = C.country_code
) T
LEFT JOIN
(
    SELECT 
        A.tmdb_id, 
        B.tmdb_title, 
        A.movie_release_date_min, 
        A.country,
        C.country_name
    FROM
    (
        SELECT tmdb_id, country, MIN(movie_release_date) AS movie_release_date_min 
        FROM movie_release
        WHERE movie_type = 4
        GROUP BY tmdb_id, country
    ) A
    LEFT JOIN ( SELECT tmdb_id, tmdb_title FROM movie_name GROUP BY tmdb_id, tmdb_title ) B
    ON A.tmdb_id = B.tmdb_id
    INNER JOIN movie_country C
    ON A.country = C.country_code
) S
ON T.tmdb_id = S.tmdb_id AND T.country = S.country
ORDER BY T.tmdb_id, T.movie_release_date_min;
"""

# 執行查詢
cursor.execute(query)

# 取得查詢結果並轉換為 DataFrame
df = pd.DataFrame(cursor.fetchall(), columns=["tmdb_id", "tmdb_title", "country_name", "theater_release", "streaming_release"])

# 關閉 cursor 和連線
cursor.close()
conn.close()

# 存成 CSV 檔案
df.to_csv("movie_release_data_min.csv", index=False, encoding='utf-8-sig')

# 顯示 DataFrame 結果
#print(df.head())

print(df)



Successfully connected!
      tmdb_id tmdb_title country_name theater_release streaming_release
0     1002317        屍房菜           台灣      2023-04-07        2024-01-24
1     1002470       四層公寓           韓國      2022-11-03              None
2     1002470       四層公寓           台灣      2023-07-07              None
3     1002470       四層公寓           法國      2024-02-21              None
4     1002535      情慾的界線           日本      2022-02-04              None
...       ...        ...          ...             ...               ...
3904   998137        填詞L           台灣      2024-03-22        2024-08-02
3905   999056         潛行           台灣      2024-01-19              None
3906   999056         潛行           韓國      2024-03-27              None
3907   999087   女鬼橋2：怨鬼樓           台灣      2023-10-13        2024-02-16
3908   999087   女鬼橋2：怨鬼樓           韓國      2023-11-15              None

[3909 rows x 5 columns]
